In [ ]:
import os
import sys
import torch
import importlib
sys.path.append(os.path.abspath('/LLM_Probing/code'))
print(sys.executable)
work_dir = '/LLM_Probing'
os.chdir(work_dir)

cache_dir = '~/.cache/huggingface/hub'

from config import ExperimentConfig
from model_loader import ModelLoader
import neuroling_probing
torch.cuda.empty_cache()
print(f"Current GPU memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Current GPU memory cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")
!nvidia-smi
!pwd

In [ ]:
import config
importlib.reload(config)
from config import ExperimentConfig

torch.cuda.empty_cache()
print(f"Current GPU memory allocated: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
print(f"Current GPU memory cached: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

hf_token = os.environ["HF_TOKEN"]
gpu_count = 1
use_gpu = True
model_name = 'Qwen/Qwen2.5-14B'
model_name_short = 'qwen2.5_14B'

config = ExperimentConfig(
    batch_size=32,
    sent_pooling='last',
    output_hidden_states=True,
    device='cuda' if torch.cuda.is_available() else 'cpu',
    task='blimp',  # {blimp, comps_base, comps_wugs}
    # residual_mode=True,
    # bow_mode=True,
    filter_mode=True,
    model_name=model_name_short,
    blimp_bow_results_path=f"{work_dir}/results/blimp_base_bow_-1.csv",
    use_memmap=True,
)
task = config.task
config


### single models

In [ ]:
loader = ModelLoader(
        model_name=model_name,
        cache_dir=cache_dir,
        hf_token=hf_token,
        use_gpu=use_gpu,
        gpu_count=gpu_count,
        speech_mode=False,
        baseline_mode=False,
        dtype=torch.bfloat16,
        # untrained=True
    )

model, tokenizer = loader.load()

In [ ]:
importlib.reload(neuroling_probing)

output_file_name = work_dir+f"/results/{task}_{model_name_short}_{str(config.pca_dim)}.csv"
print("Output file:", output_file_name)

config.update({
    'device': torch.device('cuda' if torch.cuda.is_available() and use_gpu else 'cpu'),
    'output_file_name': output_file_name,
})

task_runner = neuroling_probing.NeurolingProbing(config, model, tokenizer=tokenizer)

if task == 'blimp':
    task_runner.run_blimp('./data/BLiMP/')
elif task in {'comps_base', 'comps_wugs'}:
    task_runner.run_comps(work_dir + '/data/COMPS')
elif task == "proofwriter":
    task_runner.run_proofwriter(os.path.join(work_dir, 'data/ProofWriter/proofwriter.jsonl'))
elif task == "logic-llm":
    task_runner.run_proofwriter(os.path.join(work_dir, 'data/Logic-LLM/logic_llm.jsonl'))
elif task == "prontoqa":
    task_runner.run_prontoqa(os.path.join(work_dir, 'data/ProntoQA/prontoqa.jsonl'))
else:
    print('task not supported')

torch.cuda.empty_cache()

print("All done.")
